In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import sqlite3
from tabulate import tabulate
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Objetivo do projeto
****
Desenvolver um modelo capaz de **prever quais usuários têm maior probabilidade de fazer uma segunda compra em até 30 dias após a realização da primeira**. Segundo um estudo feito no passado, estes usuários possuem maior probabilidade de **fidelizar** na empresa e passar a realizar aquisições mensalmente.

# Conjunto de dados públicos de comércio eletrônico brasileiro por Olist
****
Este é um conjunto de dados público de comércio eletrônico brasileiro de pedidos feitos na Olist Store. O conjunto de dados contém informações de 100 mil pedidos de 2016 a 2018 feitos em vários marketplaces no Brasil. Seus recursos permitem visualizar um pedido de várias dimensões: desde o status do pedido, preço, pagamento e desempenho do frete até a localização do cliente, atributos do produto e, finalmente, avaliações escritas pelos clientes. Também lançamos um conjunto de dados de geolocalização que relaciona os códigos postais brasileiros às coordenadas lat/lng.

Estes são dados comerciais reais, foram anonimizados e as referências às empresas e parceiros no texto da revisão foram substituídas pelos nomes das grandes casas de Game of Thrones.

*url: www.kaggle.com/datasets/olistbr/brazilian-ecommerce*

## Sobre a Olist
Olist é uma plataforma de comércio eletrônico brasileira que conecta pequenas e médias empresas a grandes marketplaces, como Mercado Livre, Americanas.com, Submarino, Shoptime, Amazon, entre outros. A plataforma foi criada em 2015 e tem como objetivo ajudar os empreendedores a aumentarem suas vendas online, oferecendo uma solução completa que inclui integração com os principais marketplaces, gestão de estoque, gestão de pedidos, logística e suporte ao cliente.

# Pré-processamento dos dados
****
Nessa etapa, será realizado o pré-processamento dos dados para torná-lo adequados para o uso em um modelo de machine learning, como por exemplo, a criação de features e **target**, divisão entre a base de treino e teste, limpeza de dados. Será utilizado a Interface DB-API para banco de dados SQLite.
## Qual o cenário considerado para criar a base de dados?
**Serão coletados somente os dados da primeira compra de cada consumidor e que foram entregues com sucesso.**  
Dado que um novo cliente fez uma compra na plataforma e teve sua compra finalizada com a entrega do produto, será coletado:
- Localização do cliente;
- Reputação do(s) produto(s) comprados;    
- Reputação da categoria do(s) produto(s) comprados;
- Reputação do vendedor;
- Informações de entrega;
- Detalhes do valor da venda;
- Detalhes do pagamento;
- **[TARGET]** Flag indicando se o cliente fez uma segunda compra em até 30 dias após a primeira compra.  


**BASE DE DADOS ESPERADA AO FIM DO PRÉ-PROCESSAMENTO SQL**   

| customer_unique_id               | customer_city         | customer_state | order_review_score | mean_products_review_score | mean_category_products_review_score | mean_seller_review_score | distinct_sellers | delivery_estimated_day_difference | late_delivery | payment_installments | payment_value | freight_percent | **new_purchase_within_30_days** |
|----------------------------------|-----------------------|----------------|--------------------|----------------------------|-------------------------------------|--------------------------|------------------|-----------------------------------|---------------|----------------------|---------------|-----------------|---------------------------------|
| 861eff4711a542e4b93843c6dd7febb0 | franca                | SP             | 3.2                |3.5                         | 4.7                                 | 2.9                      | 1                | -7                                | 0             | 8                    | 99.33         | 0.15            | 1                               |
| 290c77bc529b7ac935b93aa66c333dc3 | sao bernardo do campo | SP             | 1.5                |2.2                         | 3.4                                 | 3.0                      | 2                | 3                                 | 1             | 2                    | 107.78        | 0.23            | 0                               |

## Criando e populando o banco de dados

In [2]:
conn   = sqlite3.connect('OListDB')
cursor = conn.cursor()

In [3]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        tablename = filename.replace('.csv', '').replace('olist_', '').replace('_dataset', '').upper()
        df = pd.read_csv(os.path.join(dirname, filename))
        df.to_sql(tablename, conn, if_exists='replace', index=False)  
        print(f'[OK] {filename} TO {tablename}')

[OK] olist_customers_dataset.csv TO CUSTOMERS
[OK] olist_sellers_dataset.csv TO SELLERS
[OK] olist_order_reviews_dataset.csv TO ORDER_REVIEWS
[OK] olist_order_items_dataset.csv TO ORDER_ITEMS
[OK] olist_products_dataset.csv TO PRODUCTS
[OK] olist_geolocation_dataset.csv TO GEOLOCATION
[OK] product_category_name_translation.csv TO PRODUCT_CATEGORY_NAME_TRANSLATION
[OK] olist_orders_dataset.csv TO ORDERS
[OK] olist_order_payments_dataset.csv TO ORDER_PAYMENTS


In [4]:
def get_query(SQL: str, tablefmt: str = 'github') -> str:
    query = cursor.execute(SQL)
    
    cols = [col[0] for col in query.description]
    rows = query.fetchall()
    
    return tabulate(rows, cols, tablefmt = tablefmt)


def create_table(SQL: str, tablename: str, tablefmt: str = 'github') -> str:
    try:
        cursor.execute(f'CREATE TABLE {tablename} AS {SQL}')
    except sqlite3.OperationalError:
        cursor.execute(f'DROP TABLE {tablename}')
        create_table(SQL, tablename)
    
    SQL_COUNT = f'''SELECT COUNT(1) AS QUANTITY FROM {tablename}'''
    
    return get_query(SQL_COUNT)

<h2 style="color: #CB4335">Dos 96.096 clientes, 93.099 compraram somente uma vez</h2>

In [5]:
SQL = '''
SELECT QTD_ORDER, COUNT(1) AS QTD FROM (
    SELECT CUSTOMER_UNIQUE_ID, COUNT(1) AS QTD_ORDER FROM (
        SELECT CUSTOMER_ID, CUSTOMER_UNIQUE_ID FROM CUSTOMERS
        ) 
        GROUP BY CUSTOMER_UNIQUE_ID
)
GROUP BY QTD_ORDER
'''

query = get_query(SQL)
print(query)

|   QTD_ORDER |   QTD |
|-------------|-------|
|           1 | 93099 |
|           2 |  2745 |
|           3 |   203 |
|           4 |    30 |
|           5 |     8 |
|           6 |     6 |
|           7 |     3 |
|           9 |     1 |
|          17 |     1 |


## Dados da primeira compra dos clientes considerando somente compras finalizadas (delivered)

In [6]:
# 274 dos 93.634 clientes estão repetidos porque eles compraram em mais de um vendedor na primeira compra
SQL_FIRST_ORDER = \
'''
SELECT OO.ORDER_ID             AS ORDER_ID 
     , CC.CUSTOMER_ID          AS CUSTOMER_ID
     , CO2.CUSTOMER_UNIQUE_ID  AS CUSTOMER_UNIQUE_ID
     , CC.CUSTOMER_CITY        AS CUSTOMER_CITY
     , CC.CUSTOMER_STATE       AS CUSTOMER_STATE
     , CO2.FIRST_ORDER         AS FIRST_ORDER
  FROM (
        SELECT CO.CUSTOMER_UNIQUE_ID
             , MIN(CO.ORDER_PURCHASE_TIMESTAMP) AS FIRST_ORDER
          FROM (
                SELECT C.CUSTOMER_UNIQUE_ID
                     , O.ORDER_PURCHASE_TIMESTAMP
                  FROM CUSTOMERS C
                  JOIN ORDERS O 
                    ON O.CUSTOMER_ID  = C.CUSTOMER_ID 
                   AND O.ORDER_STATUS = 'delivered'
            ) CO
        GROUP BY CO.CUSTOMER_UNIQUE_ID
        ) CO2
  JOIN CUSTOMERS CC
    ON CC.CUSTOMER_UNIQUE_ID = CO2.CUSTOMER_UNIQUE_ID 
  JOIN ORDERS OO
    ON OO.CUSTOMER_ID = CC.CUSTOMER_ID
   AND OO.ORDER_PURCHASE_TIMESTAMP = CO2.FIRST_ORDER
'''

res = create_table(SQL_FIRST_ORDER, 'MV_FIRST_ORDER')
print(res)

|   QUANTITY |
|------------|
|      93634 |


In [7]:
SQL = '''
SELECT * FROM MV_FIRST_ORDER LIMIT 5
'''

query = get_query(SQL)
print(query)

| ORDER_ID                         | CUSTOMER_ID                      | CUSTOMER_UNIQUE_ID               | CUSTOMER_CITY   | CUSTOMER_STATE   | FIRST_ORDER         |
|----------------------------------|----------------------------------|----------------------------------|-----------------|------------------|---------------------|
| e22acc9c116caa3f2b7121bbb380d08e | fadbb3709178fc513abc1b2670aa1ad2 | 0000366f3b9a7992bf8c76cfdf3221e2 | cajamar         | SP               | 2018-05-10 10:56:27 |
| 3594e05a005ac4d06a72673270ef9ec9 | 4cb282e167ae9234755102258dd52ee8 | 0000b849f77a49e4a4ce2b2a4ca5be3f | osasco          | SP               | 2018-05-07 11:11:27 |
| b33ec3b699337181488304f362a6b734 | 9b3932a6253894a02c1df9d19004239f | 0000f46a3911fa3c0805444483337064 | sao jose        | SC               | 2017-03-10 21:05:03 |
| 41272756ecddd9a9ed0180413cc22fb6 | 914991f0c02ef0843c0e7010c819d642 | 0000f6ccb0745a6a4b88665a16c9f078 | belem           | PA               | 2017-10-12 20:29:41 |
| d9

## Dados da entrega: verificar se houve atraso na entrega e diferença entre o dia da entrega e o dia estimado da entrega

In [8]:
# 21 Compras não possuem data de entrega preenchidos
SQL_ORDER_DELIVERY = \
'''
SELECT OO.ORDER_ID                                    AS ORDER_ID
     , OO.DELIVERY_ESTIMATED_DAY_DIFFERENCE           AS DELIVERY_ESTIMATED_DAY_DIFFERENCE
     , CASE                                  
       WHEN OO.DELIVERY_ESTIMATED_DAY_DIFFERENCE < 0
         THEN 1
         ELSE 0
       END                                            AS LATE_DELIVERY
  FROM (
        SELECT O.ORDER_ID
             , CAST ( JulianDay(O.ORDER_ESTIMATED_DELIVERY_DATE) - JulianDay(O.ORDER_DELIVERED_CUSTOMER_DATE) AS INTEGER) AS delivery_estimated_day_difference
          FROM ORDERS O
          JOIN MV_FIRST_ORDER FO
            ON FO.CUSTOMER_ID = O.CUSTOMER_ID
         WHERE O.ORDER_STATUS = 'delivered'
        ) OO
 
'''

res = create_table(SQL_ORDER_DELIVERY, 'MV_ORDER_DELIVERY')
print(res)

|   QUANTITY |
|------------|
|      93613 |


In [9]:
SQL = '''SELECT * FROM MV_ORDER_DELIVERY LIMIT 5'''

query = get_query(SQL)
print(query)

| ORDER_ID                         |   DELIVERY_ESTIMATED_DAY_DIFFERENCE |   LATE_DELIVERY |
|----------------------------------|-------------------------------------|-----------------|
| 53cdb2fc8bc7dce0b6741e2150273451 |                                   5 |               0 |
| 47770eb9100c2d0c44946d9cf07ec65d |                                  17 |               0 |
| 949d5b44dbf5de918fe9c16f97b45f8a |                                  12 |               0 |
| ad21c59c0840e6cb83a9ceb5573f8159 |                                   9 |               0 |
| a4591c265e18cb1dcee52889e2d8acc3 |                                   5 |               0 |


## Dados do pagamento: tipo do pagamento, número de parcelas, valor total e percentual do frete

In [10]:
# 5 Registros ficaram de fora porque não tinham informações de pagamento ou não tinham informações dos itens da compra
SQL_PAYMENTS = \
'''
SELECT PP.ORDER_ID                                 AS ORDER_ID
     , PP.PAYMENT_INSTALLMENTS                     AS PAYMENT_INSTALLMENTS
     , PP.PAYMENT_VALUE                            AS PAYMENT_VALUE
     , ROUND(I.FREIGHT_VALUE/PP.PAYMENT_VALUE, 2)  AS FREIGHT_PERCENT
  FROM (
        SELECT P.ORDER_ID                                AS ORDER_ID
             , MAX(P.PAYMENT_INSTALLMENTS)               AS PAYMENT_INSTALLMENTS
             , SUM(P.PAYMENT_VALUE)                      AS PAYMENT_VALUE
          FROM ORDER_PAYMENTS P
        GROUP BY P.ORDER_ID
       ) PP
  JOIN (
        SELECT I.ORDER_ID           AS ORDER_ID
             , SUM(I.FREIGHT_VALUE) AS FREIGHT_VALUE
          FROM ORDER_ITEMS I
        GROUP BY I.ORDER_ID
        ) I
    ON I.ORDER_ID = PP.ORDER_ID
 JOIN MV_FIRST_ORDER O 
   ON O.ORDER_ID = I.ORDER_ID
'''

res = create_table(SQL_PAYMENTS, 'MV_ORDER_PAYMENTS')
print(res)

|   QUANTITY |
|------------|
|      93629 |


In [11]:
SQL = '''SELECT * FROM MV_ORDER_PAYMENTS LIMIT 5'''

query = get_query(SQL)
print(query)

| ORDER_ID                         |   PAYMENT_INSTALLMENTS |   PAYMENT_VALUE |   FREIGHT_PERCENT |
|----------------------------------|------------------------|-----------------|-------------------|
| 00010242fe8c5a6d1ba2dd792cb16214 |                      2 |           72.19 |              0.18 |
| 00018f77f2f0320c557190d7a144bdd3 |                      3 |          259.83 |              0.08 |
| 000229ec398224ef6ca0657da4fc703e |                      5 |          216.87 |              0.08 |
| 00024acbcdf0a6daa1e931b038114c75 |                      2 |           25.78 |              0.5  |
| 00042b26cf59d7ce69dfabb4e55b4fd9 |                      3 |          218.04 |              0.08 |


## Dados da review da compra, do vendedor, do produto e da categoria do produto.


In [12]:
# 620 compras não tiveram avaliações
SQL_ORDER_SCORE = \
'''
SELECT R.ORDER_ID, R.REVIEW_SCORE
  FROM ORDER_REVIEWS R
  JOIN MV_FIRST_ORDER FO
    ON FO.ORDER_ID = R.ORDER_ID
'''

res = create_table(SQL_ORDER_SCORE, 'MV_ORDER_REVIEW_SCORE')
print(res)

|   QUANTITY |
|------------|
|      93378 |


In [13]:
SQL = '''
SELECT * FROM MV_ORDER_REVIEW_SCORE LIMIT 5
'''

query = get_query(SQL)
print(query)

| order_id                         |   review_score |
|----------------------------------|----------------|
| 73fc7af87114b39712e6da79b0a377eb |              4 |
| a548910a1c6147796b98fdf73dbeba33 |              5 |
| f9e4b658b201a9f2ecdecbb34bed034b |              5 |
| 658677c97b385a9be170737859d3511b |              5 |
| 8e6bfb81e283fa7e4f11123a3fb894f1 |              5 |


In [14]:
# 14 compras foram realizadas com mais de um vendedor
SQL_SELLER_SCORE = \
'''
SELECT I.SELLER_ID                   AS SELLER_ID
     , ROUND(AVG(R.REVIEW_SCORE), 1) AS MEAN_SELLER_REVIEW_SCORE
  FROM ORDER_REVIEWS R
  JOIN ORDERS O 
    ON O.ORDER_ID = R.ORDER_ID
  JOIN ORDER_ITEMS I
    ON I.ORDER_ID = R.ORDER_ID
  JOIN MV_FIRST_ORDER FO
    ON FO.ORDER_ID = R.ORDER_ID
 GROUP BY I.SELLER_ID
'''

res = create_table(SQL_SELLER_SCORE, 'MV_SELLER_REVIEW_SCORE')
print(res)

|   QUANTITY |
|------------|
|       2951 |


In [15]:
SQL = '''
SELECT * FROM MV_SELLER_REVIEW_SCORE LIMIT 5
'''

query = get_query(SQL)
print(query)

| SELLER_ID                        |   MEAN_SELLER_REVIEW_SCORE |
|----------------------------------|----------------------------|
| 0015a82c2db000af6aaaf3ae2ecb0532 |                        3.7 |
| 001cca7ae9ae17fb1caed9dfb1094831 |                        4   |
| 002100f778ceb8431b7a1020ff7ab48f |                        4.1 |
| 003554e2dce176b5555353e4f3555ac8 |                        5   |
| 004c9cd9d87a3c30c522c48c4fc07416 |                        4.1 |


In [16]:
SQL_PRODUCT_SCORE = \
'''
SELECT I.PRODUCT_ID                   AS PRODUCT_ID
     , ROUND(AVG(R.REVIEW_SCORE), 1) AS MEAN_PRODUCT_REVIEW_SCORE
  FROM ORDER_REVIEWS R
  JOIN ORDERS O 
    ON O.ORDER_ID = R.ORDER_ID
  JOIN ORDER_ITEMS I
    ON I.ORDER_ID = R.ORDER_ID
  JOIN MV_FIRST_ORDER FO
    ON FO.ORDER_ID = R.ORDER_ID
 GROUP BY I.PRODUCT_ID
'''

res = create_table(SQL_PRODUCT_SCORE, 'MV_PRODUCT_REVIEW_SCORE')
print(res)

|   QUANTITY |
|------------|
|      31384 |


In [17]:
SQL = '''
SELECT * FROM MV_PRODUCT_REVIEW_SCORE LIMIT 5
'''

query = get_query(SQL)
print(query)

| PRODUCT_ID                       |   MEAN_PRODUCT_REVIEW_SCORE |
|----------------------------------|-----------------------------|
| 00066f42aeeb9f3007548bb9d3f33c38 |                           5 |
| 00088930e925c41fd95ebfe695fd2655 |                           4 |
| 0009406fd7479715e4bef61dd91f2462 |                           1 |
| 000b8f95fcb9e0096488278317764d19 |                           5 |
| 000d9be29b5207b54e86aa1b1ac54872 |                           5 |


In [18]:
SQL_PRODUCT_CATEGORY_SCORE = \
'''
SELECT PC.PRODUCT_CATEGORY_NAME                     AS PRODUCT_CATEGORY_NAME
     , ROUND(AVG(PC.MEAN_PRODUCT_REVIEW_SCORE ), 1) AS MEAN_PRODUCT_CATEGORY_REVIEW_SCORE
  FROM (
        SELECT I.PRODUCT_ID                   AS PRODUCT_ID
             , P.PRODUCT_CATEGORY_NAME        AS PRODUCT_CATEGORY_NAME
             , ROUND(AVG(R.REVIEW_SCORE), 1)  AS MEAN_PRODUCT_REVIEW_SCORE
          FROM ORDER_REVIEWS R
          JOIN ORDERS O 
            ON O.ORDER_ID = R.ORDER_ID
          JOIN ORDER_ITEMS I
            ON I.ORDER_ID = R.ORDER_ID
          JOIN PRODUCTS P
            ON P.PRODUCT_ID = I.PRODUCT_ID
          JOIN MV_FIRST_ORDER FO
            ON FO.ORDER_ID = R.ORDER_ID
         GROUP BY I.PRODUCT_ID
                , P.PRODUCT_CATEGORY_NAME
        ) PC
GROUP BY PC.PRODUCT_CATEGORY_NAME
'''

res = create_table(SQL_PRODUCT_CATEGORY_SCORE, 'MV_PRODUCT_CATEGORY_REVIEW_SCORE')
print(res)

|   QUANTITY |
|------------|
|         74 |


In [19]:
SQL = '''
SELECT * FROM MV_PRODUCT_CATEGORY_REVIEW_SCORE LIMIT 5
'''

query = get_query(SQL)
print(query)

| PRODUCT_CATEGORY_NAME     |   MEAN_PRODUCT_CATEGORY_REVIEW_SCORE |
|---------------------------|--------------------------------------|
|                           |                                  4   |
| agro_industria_e_comercio |                                  4   |
| alimentos                 |                                  4.3 |
| alimentos_bebidas         |                                  4.5 |
| artes                     |                                  4.3 |


## Contagem de vendedores distintos por compra

In [20]:
SQL_DISTINCT_SELLERS = \
'''
SELECT FO.CUSTOMER_UNIQUE_ID AS CUSTOMER_UNIQUE_ID
     , COUNT(1)              AS DISTINCT_SELLERS 
 FROM MV_FIRST_ORDER FO
GROUP BY FO.CUSTOMER_UNIQUE_ID
'''

res = create_table(SQL_DISTINCT_SELLERS, 'MV_DISTINCT_SELLERS')
print(res)

|   QUANTITY |
|------------|
|      93358 |


In [21]:
SQL = '''
SELECT * FROM MV_DISTINCT_SELLERS 
WHERE DISTINCT_SELLERS > 1
LIMIT 5
'''

query = get_query(SQL)
print(query)

| CUSTOMER_UNIQUE_ID               |   DISTINCT_SELLERS |
|----------------------------------|--------------------|
| 02b20b7c813efede140142ac610e36dc |                  2 |
| 031e19fc630c4121f1238716f41675c3 |                  2 |
| 050555c26479bbb8ba0740c37e7454d0 |                  2 |
| 06a52782a04f0086d16b9c22d0e29438 |                  2 |
| 0710e0c85fe7cb494d624e0863782e46 |                  2 |


## Obter os clientes que fizeram uma nova compra em até 30 dias

In [22]:
SQL_SEC_ORDER_WITHIN_30 = \
'''
SELECT SO.CUSTOMER_UNIQUE_ID                                                                          AS CUSTOMER_UNIQUE_ID
     , CASE WHEN CAST ( JulianDay(SO.SECOND_ORDER) - JulianDay(SO.FIRST_ORDER) AS INTEGER) <= 30
         THEN 1
         ELSE 0
       END                                                                                            AS SECOND_ORDER_WITHIN_30_DAYS  
  FROM (
        SELECT FO.CUSTOMER_UNIQUE_ID           AS CUSTOMER_UNIQUE_ID 
             , FO.FIRST_ORDER                  AS FIRST_ORDER
             , MIN(O.ORDER_PURCHASE_TIMESTAMP) AS SECOND_ORDER
          FROM ORDERS O
          JOIN CUSTOMERS C
            ON O.CUSTOMER_ID = C.CUSTOMER_ID
          JOIN MV_FIRST_ORDER FO
            ON FO.CUSTOMER_UNIQUE_ID = C.CUSTOMER_UNIQUE_ID
           AND FO.FIRST_ORDER <> O.ORDER_PURCHASE_TIMESTAMP
        GROUP BY FO.CUSTOMER_UNIQUE_ID
) SO
'''

res = create_table(SQL_SEC_ORDER_WITHIN_30, 'MV_SECOND_ORDER_WITHIN_30_DAYS')
print(res)

|   QUANTITY |
|------------|
|       2725 |


In [43]:
SQL = '''
SELECT * FROM MV_SECOND_ORDER_WITHIN_30_DAYS SO30 LIMIT 5
'''

query = get_query(SQL)
print(query)

| CUSTOMER_UNIQUE_ID               |   SECOND_ORDER_WITHIN_30_DAYS |
|----------------------------------|-------------------------------|
| 00172711b30d52eea8b313a7f2cced02 |                             1 |
| 004288347e5e88a27ded2bb23747066c |                             0 |
| 004b45ec5c64187465168251cd1c9c2f |                             1 |
| 0058f300f57d7b93c477a131a59b36c3 |                             1 |
| 00a39521eb40f7012db50455bf083460 |                             1 |


In [23]:
SQL = '''
SELECT SECOND_ORDER_WITHIN_30_DAYS, COUNT(1) AS "CUSTOMERS_WHO_PURCHASED_MORE_THAN_ONCE" FROM MV_SECOND_ORDER_WITHIN_30_DAYS SO30 GROUP BY SECOND_ORDER_WITHIN_30_DAYS
'''

query = get_query(SQL)
print(query)

|   SECOND_ORDER_WITHIN_30_DAYS |   CUSTOMERS_WHO_PURCHASED_MORE_THAN_ONCE |
|-------------------------------|------------------------------------------|
|                             0 |                                     1420 |
|                             1 |                                     1305 |


## Unindo dataset

In [69]:
DATASET = \
'''
SELECT    D.CUSTOMER_CITY
        , D.CUSTOMER_STATE
        , D.DISTINCT_SELLERS
        , D.DELIVERY_ESTIMATED_DAY_DIFFERENCE
        , D.LATE_DELIVERY
        , D.PAYMENT_INSTALLMENTS
        , D.PAYMENT_VALUE
        , D.FREIGHT_PERCENT
        , D.ORDER_REVIEW_SCORE
        , D.MEAN_SELLER_REVIEW_SCORE
        , D.MEAN_PRODUCT_REVIEW_SCORE
        , D.MEAN_PRODUCT_CATEGORY_REVIEW_SCORE
        , D.SECOND_ORDER_WITHIN_30_DAYS
FROM (
        SELECT FO.CUSTOMER_UNIQUE_ID                            AS CUSTOMER_UNIQUE_ID
             , MAX(FO.CUSTOMER_CITY)                            AS CUSTOMER_CITY
             , MAX(FO.CUSTOMER_STATE)                           AS CUSTOMER_STATE
             , AVG(DS.DISTINCT_SELLERS)                         AS DISTINCT_SELLERS
             , AVG(OD.DELIVERY_ESTIMATED_DAY_DIFFERENCE)        AS DELIVERY_ESTIMATED_DAY_DIFFERENCE
             , AVG(OD.LATE_DELIVERY)                            AS LATE_DELIVERY
             , AVG(OP.PAYMENT_INSTALLMENTS)                     AS PAYMENT_INSTALLMENTS
             , AVG(OP.PAYMENT_VALUE)                            AS PAYMENT_VALUE
             , AVG(OP.FREIGHT_PERCENT)                          AS FREIGHT_PERCENT
             , AVG(OS.REVIEW_SCORE)                             AS ORDER_REVIEW_SCORE
             , AVG(SS.MEAN_SELLER_REVIEW_SCORE)                 AS MEAN_SELLER_REVIEW_SCORE
             , AVG(PS.MEAN_PRODUCT_REVIEW_SCORE)                AS MEAN_PRODUCT_REVIEW_SCORE
             , AVG(CS.MEAN_PRODUCT_CATEGORY_REVIEW_SCORE)       AS MEAN_PRODUCT_CATEGORY_REVIEW_SCORE
             , IFNULL(SO30.SECOND_ORDER_WITHIN_30_DAYS, 0)      AS SECOND_ORDER_WITHIN_30_DAYS
        FROM MV_FIRST_ORDER FO
        LEFT JOIN MV_DISTINCT_SELLERS DS
          ON FO.CUSTOMER_UNIQUE_ID = DS.CUSTOMER_UNIQUE_ID
        LEFT JOIN MV_ORDER_DELIVERY OD
          ON FO.ORDER_ID = OD.ORDER_ID
        LEFT JOIN MV_ORDER_PAYMENTS OP
          ON FO.ORDER_ID = OP.ORDER_ID
        LEFT JOIN MV_ORDER_REVIEW_SCORE OS
          ON FO.ORDER_ID = OS.ORDER_ID
        JOIN ORDER_ITEMS I
          ON FO.ORDER_ID = I.ORDER_ID
        LEFT JOIN MV_SELLER_REVIEW_SCORE SS
          ON SS.SELLER_ID = I.SELLER_ID
        LEFT JOIN MV_PRODUCT_REVIEW_SCORE PS
          ON PS.PRODUCT_ID = I.PRODUCT_ID
        JOIN PRODUCTS P
          ON P.PRODUCT_ID = I.PRODUCT_ID
        LEFT JOIN MV_PRODUCT_CATEGORY_REVIEW_SCORE CS
          ON CS.PRODUCT_CATEGORY_NAME = P.PRODUCT_CATEGORY_NAME
        LEFT JOIN MV_SECOND_ORDER_WITHIN_30_DAYS SO30
          ON FO.CUSTOMER_UNIQUE_ID = SO30.CUSTOMER_UNIQUE_ID
        GROUP BY FO.CUSTOMER_UNIQUE_ID
) D
'''

res = create_table(DATASET, 'DATASET')
print(res)

|   QUANTITY |
|------------|
|      93358 |


In [70]:
DATASET = '''
SELECT * FROM DATASET D LIMIT 5
'''

query = get_query(DATASET)
print(query)

| CUSTOMER_CITY   | CUSTOMER_STATE   |   DISTINCT_SELLERS |   DELIVERY_ESTIMATED_DAY_DIFFERENCE |   LATE_DELIVERY |   PAYMENT_INSTALLMENTS |   PAYMENT_VALUE |   FREIGHT_PERCENT |   ORDER_REVIEW_SCORE |   MEAN_SELLER_REVIEW_SCORE |   MEAN_PRODUCT_REVIEW_SCORE |   MEAN_PRODUCT_CATEGORY_REVIEW_SCORE |   SECOND_ORDER_WITHIN_30_DAYS |
|-----------------|------------------|--------------------|-------------------------------------|-----------------|------------------------|-----------------|-------------------|----------------------|----------------------------|-----------------------------|--------------------------------------|-------------------------------|
| cajamar         | SP               |                  1 |                                   4 |               0 |                      8 |          141.9  |              0.08 |                    5 |                        4.1 |                         4.2 |                                  3.9 |                             0 |
| os

In [73]:
DATASET = '''
SELECT * FROM DATASET D LIMIT 5
'''

query = get_query(DATASET)
print(query)

| CUSTOMER_CITY   | CUSTOMER_STATE   |   DISTINCT_SELLERS |   DELIVERY_ESTIMATED_DAY_DIFFERENCE |   LATE_DELIVERY |   PAYMENT_INSTALLMENTS |   PAYMENT_VALUE |   FREIGHT_PERCENT |   ORDER_REVIEW_SCORE |   MEAN_SELLER_REVIEW_SCORE |   MEAN_PRODUCT_REVIEW_SCORE |   MEAN_PRODUCT_CATEGORY_REVIEW_SCORE |   SECOND_ORDER_WITHIN_30_DAYS |
|-----------------|------------------|--------------------|-------------------------------------|-----------------|------------------------|-----------------|-------------------|----------------------|----------------------------|-----------------------------|--------------------------------------|-------------------------------|
| cajamar         | SP               |                  1 |                                   4 |               0 |                      8 |          141.9  |              0.08 |                    5 |                        4.1 |                         4.2 |                                  3.9 |                             0 |
| os

In [72]:
df = pd.read_sql('SELECT * FROM DATASET', conn)
df.head()

,CUSTOMER_CITY,CUSTOMER_STATE,DISTINCT_SELLERS,DELIVERY_ESTIMATED_DAY_DIFFERENCE,LATE_DELIVERY,PAYMENT_INSTALLMENTS,PAYMENT_VALUE,FREIGHT_PERCENT,ORDER_REVIEW_SCORE,MEAN_SELLER_REVIEW_SCORE,MEAN_PRODUCT_REVIEW_SCORE,MEAN_PRODUCT_CATEGORY_REVIEW_SCORE,SECOND_ORDER_WITHIN_30_DAYS
0,cajamar,SP,1.0,4.0,0.0,8.0,141.90,0.08,5.0,4.1,4.2,3.9,0
1,osasco,SP,1.0,4.0,0.0,1.0,27.19,0.30,4.0,4.0,4.7,4.2,0
2,sao jose,SC,1.0,1.0,0.0,8.0,86.22,0.20,3.0,4.1,4.6,4.3,0
3,belem,PA,1.0,11.0,0.0,4.0,43.62,0.40,4.0,3.9,4.7,4.0,0
4,sorocaba,SP,1.0,7.0,0.0,6.0,196.89,0.09,5.0,3.8,5.0,4.0,0


# Análise exploratória dos dados
****
*forneça uma breve análise exploratória dos dados. Isso pode incluir a distribuição das classes de destino, uma matriz de correlação entre as variáveis, visualizações de distribuições, entre outros*

# Modelagem
****
*explique o modelo que você escolheu e por que o escolheu. Explique também os hiperparâmetros que você usou.*

# Resultados
****
*apresente os resultados do seu modelo, como a acurácia ou a precisão. Você pode incluir também outras métricas que sejam relevantes para o seu problema.*

# Conclusão
****
*faça uma conclusão sobre o seu projeto, destacando seus resultados e eventuais limitações ou desafios que você enfrentou durante o processo.*  
Neste projeto tive a limitação de não trabalhar com o PL/SQL. O que dificultou no desenvolvimento das métricas na etapa de pré-processamento dos dados. No uso desse modelo na vida real, a escrita de código procedural no banco de dados com o PL/SQL seria essencial para executar tarefas mais complexas, como processamento de dados, lógica de negócios, manipulação de exceções, entre outras coisas.